In [59]:
# 라이브러리 불러오기
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup as bs4
import pprint
import warnings
import re
import string
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 크롬 옵션 객체 생성
options = Options()

#지정한 user-agent로 설정합니다.
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36")

options.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')
options.add_argument('--start-maximized')

# 경고 무시
warnings.filterwarnings(action='ignore')

In [89]:
url = 'https://www.cheonan.go.kr/prog/mayorWish/mayor/sub03_01/list.do'
driver = webdriver.Chrome('/Users/kanovatio/chromedriver', options=options)
driver.get(url)

today_exit = driver.find_element(By.XPATH, '//*[@id="exitDay"]')
today_exit.click()
x_button = driver.find_element(By.XPATH,'//*[@id="exitThisLayer"]')
x_button.click()

# 특수문자를 포함한 문자열
special_characters = string.punctuation

claim_list = []
interrupted = False

while True:
    for i in range(1, 12):
        if i + 2 == 13:
            html = driver.page_source
            soup = bs4(html, 'html.parser')
            table = soup.find('div', {'class': 'table-responsive'})
            tbody = table.find('tbody')

            for tr in tbody.find_all('tr'):
                tds = tr.find_all('td')
                if tds:
                    claim_list.append({
                        '민원 내용': tds[3].text.replace('\n', '').replace('\r', '').lstrip()
                    })

            # 10*n 페이지로 넘김
            try:
                next_page_button = driver.find_element(By.XPATH, '//*[@id="txt"]/div[2]/div[4]/li[13]/a')
                next_page_button.click()
                
            except:
                interrupted = True
                break

        else :
            html = driver.page_source
            soup = bs4(html, 'html.parser')
            table = soup.find('div', {'class': 'table-responsive'})
            tbody = table.find('tbody')

            for tr in tbody.find_all('tr'):
                tds = tr.find_all('td')
                if tds:
                    claim_list.append({
                        '민원 내용': tds[3].text.replace('\n', '').replace('\r', '').lstrip()
                    })

            # 다음 페이지로 이동
            try :
                next_button = driver.find_element(By.XPATH, f'//*[@id="txt"]/div[2]/div[4]/li[{i + 2}]')
                next_button.click()
                time.sleep(1)
                
            except :
                interrupted = True
                break
    
    if interrupted:
        break
        
# 데이터프레임 생성
claim_data = pd.DataFrame(data=claim_list)

for sc in special_characters :
    claim_data['민원 내용'] = claim_data['민원 내용'].str.replace(sc, "")

claim_data

,민원 내용
0,안전신문고 주정차 관련 문제 해결 부탁드립니다 ...
1,성성물빛호수공원에 발전을 바라며
2,신방동 체육시설
3,시내버스
4,전기차보조금
...,...
1833,전기차 수요성이 높은 천안의 위치 보조금 지원 확대 요청드립니다 ...
1834,안녕하세요 담당자님
1835,돈워리
1836,부친 집을 지나가는 도로의 보상처리에 대한 분발을 요청드리고자 시장님께 부탁드립니다...


In [91]:
# 데이터 저장
claim_data.to_csv('claim_data.csv', index = True, encoding='cp949')